In [1]:
import pandas as pd
from collections import defaultdict
import mysql.connector
import glob
import os
pd.set_option('display.max_columns', 10)

In [2]:
conn = mysql.connector.connect(user='root', password='password', host='127.0.0.1', port='3306',
                               database='expression_atlas_shotgun')
mycursor = conn.cursor()

# check the connection
if conn.is_connected():
    print("connection succesfull")
else:
    print("no connection")

connection succesfull


In the downloadable links from the human protein atlas, normal_tissue.tsv was selected.
This file contains expression profiles for proteins in human tissues based on immunohistochemisty using tissue micro arrays.
The tab-separated file includes Ensembl gene identifier ("Gene"), tissue name ("Tissue"), annotated cell type ("Cell type"),
expression value ("Level"), and the gene reliability of the expression value ("Reliability").
The data is based on The Human Protein Atlas version 20.0 and Ensembl version 92.38.
Expression values are not detected, low, medium and high, ordinal values which will be replaced with 0, 1, 2 and 3
Levels that were N/A or Not representative were removed
Uniprot gene namedss needed to be mapped to UniProt ID's

In [2]:
hpa = pd.read_csv('/home/compomics/git/tissue-prediction/Tables/normal_tissue.tsv', sep='\t')
hpa.head()

FileNotFoundError: [Errno 2] File /home/compomics/git/tissue-prediction/Tables/normal_tissue.tsv does not exist: '/home/compomics/git/tissue-prediction/Tables/normal_tissue.tsv'

In [30]:
hpa['Level'].unique()

array(['Not detected', 'Medium', 'High', 'Low', nan, 'Not representative'],
      dtype=object)

In [31]:
levels = ['Not detected', 'Low', 'Medium', 'High']
indexlevels = hpa[hpa['Level'] == 'Not representative'].index
hpa.drop(indexlevels, inplace=True)
hpa.dropna(subset=['Level'], inplace=True)

In [32]:
hpa['Level'].unique()

array(['Not detected', 'Medium', 'High', 'Low'], dtype=object)

In [33]:
# replace ordinal values by integers
hpa['Level'].replace({'Not detected': 0, 'Low': 1, 'Medium': 2, 'High': 3}, inplace=True)
hpa['Level'].unique()

array([0, 2, 3, 1])

In [34]:
hpa.to_csv('hpa.csv', index=False)


Replace Uniprot Gene Names by Uniprot IDs

In [43]:
gene_names = hpa['Gene name'].tolist()
gene_names = list(dict.fromkeys(gene_names))
gene_names_str = ' '.join(gene_names)
file = open('gene_names_file.txt', 'w')
n = file.write(gene_names_str)
file.close()



Retrieve ID mapping tool from Uniprot was used. Gene names txt file was uploaded, filtered on Homo Sapiens
and swissprot entries (reviewed).
The results was downloaded as uniprot-yourlist.tab


In [56]:
uni_map = pd.read_csv('uniprot-yourlist.tab', sep='\t')
uni_map = uni_map.rename(columns={'yourlist:M20201127A94466D2655679D1FD8953E075198DA818A0DAG': 'Gene name'})
uni_map.head()

,Entry,Gene name
0,O43657,TSPAN6
1,O60762,DPM1
2,Q8IZE3,SCYL3
3,Q9NSG2,C1orf112
4,P09769,FGR


In [57]:
protein_to_tissue = pd.merge(hpa, uni_map, on='Gene name')
protein_to_tissue = protein_to_tissue.drop(columns=['Gene', 'Gene name', 'Reliability'])
protein_to_tissue.head()


,Tissue,Cell type,Level,Entry
0,adipose tissue,adipocytes,0,O43657
1,adrenal gland,glandular cells,0,O43657
2,appendix,glandular cells,2,O43657
3,appendix,lymphoid tissue,0,O43657
4,bone marrow,hematopoietic cells,0,O43657


In [58]:
protein_to_tissue = protein_to_tissue[['Entry', 'Tissue', 'Cell type', 'Level' ]]
protein_to_tissue = protein_to_tissue.rename(columns={'Entry': 'uniprot_id'})
protein_to_tissue.head(35)

,uniprot_id,Tissue,Cell type,Level
0,O43657,adipose tissue,adipocytes,0
1,O43657,adrenal gland,glandular cells,0
2,O43657,appendix,glandular cells,2
3,O43657,appendix,lymphoid tissue,0
4,O43657,bone marrow,hematopoietic cells,0
5,O43657,breast,adipocytes,0
6,O43657,breast,glandular cells,3
7,O43657,breast,myoepithelial cells,0
8,O43657,bronchus,respiratory epithelial cells,3
9,O43657,caudate,glial cells,0


In [64]:
# one protein can be multiple times present in the same tissue, but in different cell types
# there are dupliacte entries endometrium 1 and 2, skin 1 and 2, soft tissue and stomach
duplicates = ['endometrium 2', 'skin 2', 'soft tissue 2', 'stomach 2']
indextissues = protein_to_tissue[protein_to_tissue['Tissue'].isin(duplicates)].index
protein_to_tissue.drop(indextissues, inplace=True)

In [66]:
protein_to_tissue['Tissue'].replace({'endometrium 1': 'endometrium',  'skin 2': 'skin', 'soft tissue 2': 'soft tissue', 'stomach 2': 'stomach'}, inplace=True)

In [68]:
protein_to_tissue.to_csv('tissue_to_protein.csv', index=False)


### Based on the tissue, an organ_id was added. A dictionary containing the tissue and its organ id can be found in tissue_to_organ_dict.txt

This organid was manually added added to the tissue_to_protein file (tissue_to_protein_organupdate.csv) and will also be added to the tissue table in the database

In [12]:
import ast 
with open('tissue_to_organ_dict.txt') as f:
    organ_dict = f.readlines()
    organ_dict = organ_dict[0]
    organ_dict = ast.literal_eval(organ_dict)

In [15]:
export = pd.read_csv('/home/compomics/git/Tissue_specific/Shotgun_atlas/tissue_export.csv', sep=',')
export.head()

,tissue_id,tissue_name,cell_type,disease_status,organ_id,fluid
0,557,Urine,Urine,Extracellular vesicles of the tissue were isol...,NaN,1
1,559,Urine,Urine,Diseased,NaN,1
2,589,Urine,Urine,No distinct annotation,NaN,1
3,599,Urine,Urine,Cancer,NaN,1
4,499,Urine,Urine,Healthy,NaN,1


In [16]:
import numpy as np
print(np.unique(export['tissue_name']))

['Adipose tissue' 'Adrenal gland' 'Ascites' 'B-cells' 'Blood' 'Bone'
 'Brain' 'Breast' 'Bronchoalveolar lavage' 'Cartilage'
 'Cerebrospinal fluid' 'Cervix' 'Colon' 'Dental plaque' 'Esophagus' 'Eye'
 'Follicular fluid' 'Gall bladder' 'Heart' 'Hela' 'Kidney' 'Liver' 'Lung'
 'Lymph node' 'Monocytes' 'NK-cells' 'Nasal polyps' 'Ovary' 'PBMC'
 'Palatine tonsils' 'Pancreas' 'Parotid gland' 'Peritoneal dialysis fluid'
 'Placenta' 'Prostate' 'Rectum' 'Retina' 'Saliva' 'Skeletal muscle' 'Skin'
 'Small intestine' 'Spinal cord' 'Sputum' 'Stomach' 'Synovial tissue'
 'T-cells' 'Testis' 'Tooth' 'Trophoblast' 'Umbilical cord' 'Unknown'
 'Ureter' 'Urinary bladder' 'Urine']


In [17]:
export['organ_id'] = export.apply(lambda row: organ_dict[row.tissue_name], axis=1)

In [18]:
export

,tissue_id,tissue_name,cell_type,disease_status,organ_id,fluid
0,557,Urine,Urine,Extracellular vesicles of the tissue were isol...,64,1
1,559,Urine,Urine,Diseased,64,1
2,589,Urine,Urine,No distinct annotation,64,1
3,599,Urine,Urine,Cancer,64,1
4,499,Urine,Urine,Healthy,64,1
...,...,...,...,...,...,...
147,518,B-cells,B-cells,Healthy,4,0
148,459,B-cells,B-cell,Healthy,4,0
149,600,Ascites,Ascites,Cancer,65,1
150,458,Adrenal gland,Adrenalgland,Healthy,2,0


In [20]:
export.to_csv('tissue_export_organupdate.csv', index=False, header=False, sep=',')

In [6]:
pd.read_csv('tissue_export_organupdate.csv')

,557,Urine,Urine.1,Extracellular vesicles of the tissue were isolated,64,1
0,559,Urine,Urine,Diseased,64,1
1,589,Urine,Urine,No distinct annotation,64,1
2,599,Urine,Urine,Cancer,64,1
3,499,Urine,Urine,Healthy,64,1
4,520,Urinary bladder,Urinary bladder,Cancer,63,0
...,...,...,...,...,...,...
146,518,B-cells,B-cells,Healthy,4,0
147,459,B-cells,B-cell,Healthy,4,0
148,600,Ascites,Ascites,Cancer,65,1
149,458,Adrenal gland,Adrenalgland,Healthy,2,0


In [7]:
import csv
tissue_organ_csv = csv.reader(open('tissue_export_organupdate.csv'), delimiter=',') #MANUALLY REMOVE HEADER
for row in tissue_organ_csv:
    tissue_organ_insert = "INSERT INTO tissue(tissue_id, tissue_name, cell_type, disease_status, organ_id, fluid) VALUES(%s,%s,%s,%s,%s,%s) ON DUPLICATE KEY UPDATE tissue_id=tissue_id, tissue_name=tissue_name, cell_type=cell_type, disease_status=disease_status, organ_id=organ_id, fluid=fluid"
    mycursor.execute(tissue_organ_insert, row) 
    conn.commit()
print('Done')

Done


In [3]:
import csv
tissue_protein_csv = csv.reader(open('/home/compomics/git/Tissue_specific/Tables/tissue_to_protein_organupdate.csv'), delimiter=';') #MANUALLY REMOVE HEADER
for row in tissue_protein_csv:
    disable = "SET FOREIGN_KEY_CHECKS=0"
    mycursor.execute(disable) 
    tissue_protein_insert = "INSERT INTO tissue_to_protein(uniprot_id, tissue, cell_type, level, organ_id) VALUES(%s,%s,%s,%s,%s) ON DUPLICATE KEY UPDATE uniprot_id = uniprot_id, organ_id=organ_id, tissue=tissue, cell_type=cell_type, level=level"
    mycursor.execute(tissue_protein_insert, row) 
    conn.commit()
print('Done')

Done
